In [705]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split

In [706]:
imdb_df = pd.read_csv("new_cleaned.csv")

In [707]:
imdb_df.shape

(714, 13)

In [708]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714 entries, 0 to 713
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   released_year  714 non-null    object 
 1   certificate    714 non-null    object 
 2   runtime        714 non-null    int64  
 3   genre          714 non-null    object 
 4   imdb_rating    714 non-null    float64
 5   meta_score     714 non-null    float64
 6   director       714 non-null    object 
 7   star1          714 non-null    object 
 8   star2          714 non-null    object 
 9   star3          714 non-null    object 
 10  star4          714 non-null    object 
 11  no_of_votes    714 non-null    int64  
 12  gross          714 non-null    float64
dtypes: float64(3), int64(2), object(8)
memory usage: 72.6+ KB


In [709]:
imdb_df.head()

,released_year,certificate,runtime,genre,imdb_rating,meta_score,director,star1,star2,star3,star4,no_of_votes,gross
0,1994,A,142,Drama,9.3,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,28341469.0
1,1972,A,175,"Crime, Drama",9.2,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411.0
2,2008,UA,152,"Action, Crime, Drama",9.0,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444.0
3,1974,A,202,"Crime, Drama",9.0,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,57300000.0
4,1957,U,96,"Crime, Drama",9.0,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,4360000.0


In [744]:
numerical = imdb_df.select_dtypes(["int64","float64"])
print(numerical.head())

   runtime  imdb_rating  meta_score  no_of_votes        gross
0      142          9.3        80.0      2343110   28341469.0
1      175          9.2       100.0      1620367  134966411.0
2      152          9.0        84.0      2303232  534858444.0
3      202          9.0        90.0      1129952   57300000.0
4       96          9.0        96.0       689845    4360000.0


In [746]:
categorical=  imdb_df.drop(numerical,axis = 1)
print(categorical.head())

  released_year certificate                 genre              director  \
0          1994           A                 Drama        Frank Darabont   
1          1972           A          Crime, Drama  Francis Ford Coppola   
2          2008          UA  Action, Crime, Drama     Christopher Nolan   
3          1974           A          Crime, Drama  Francis Ford Coppola   
4          1957           U          Crime, Drama          Sidney Lumet   

            star1           star2          star3           star4  
0     Tim Robbins  Morgan Freeman     Bob Gunton  William Sadler  
1   Marlon Brando       Al Pacino     James Caan    Diane Keaton  
2  Christian Bale    Heath Ledger  Aaron Eckhart   Michael Caine  
3       Al Pacino  Robert De Niro  Robert Duvall    Diane Keaton  
4     Henry Fonda     Lee J. Cobb  Martin Balsam    John Fiedler  


In [748]:
#And also lets define our target.
target = imdb_df['imdb_rating']
features = imdb_df.drop(target,axis=1)

KeyError: '[9.3, 9.2, 9.0, 9.0, 9.0, 8.9, 8.9, 8.9, 8.8, 8.8, 8.8, 8.8, 8.8, 8.7, 8.7, 8.7, 8.7, 8.7, 8.6, 8.6, 8.6, 8.6, 8.6, 8.6, 8.6, 8.6, 8.6, 8.6, 8.6, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.4, 8.4, 8.4, 8.4, 8.4, 8.4, 8.4, 8.4, 8.4, 8.4, 8.4, 8.4, 8.4, 8.4, 8.4, 8.4, 8.4, 8.4, 8.4, 8.4, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.3, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1, 8.1

In [713]:
# Now we perform the division between Train and Test, we will reserve 20% of our data to Test.
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

##  Regression

In [714]:
# we will use KNN Regressor, we will use hyperparameter n_neighbors =10 .

In [715]:
from sklearn.neighbors import KNeighborsRegressor

In [716]:
knn = KNeighborsRegressor(n_neighbors=10) # K=10

In [ ]:
knn.fit(X_train, y_train)

In [718]:
#We are going to evaluate our model performance with R-Squared
print(f"The R2 of the model is {knn.score(X_test, y_test): .2f}")

The R2 of the model is  0.01
